In [1]:
from bs4 import BeautifulSoup
import requests

res = {'title': [], 'date': [], 'url': []}
html = requests.get("https://sw7up.cbnu.ac.kr/community/notice?page=1&limit=10&sort=-createdAt").text
bsObject = BeautifulSoup(html, "html.parser")

for link in bsObject.find_all('span', 'mb-0'):
    print(str(link).split('>')[1].split('<')[0])
    res['title'].append(str(link).split('>')[1].split('<')[0])

2022년 SW중심대학사업단 Git-Github 기초/심화 특강 신청 안내(~11/24)
2022학년도 2학기 마일리지 장학금 제출 안내(필독★)
2022-1기 일경험수련과정 참여신청서 제출안내
2022년 SW중심대학사업단 글로벌캡스톤디자인(CEDC 2022) 참가 신청 안내(~11/18)
코딩이력관리시스템 매뉴얼 다운로드
2022년 2차 Solve-UpⓇ 프로그래밍 경진대회 참가 신청 안내(~11/23)
2022학년도 2학기 마일리지 장학생 모집 안내 (신청일: 11/14~11/29)
(홍보) KOSA, 미래 유망 산업 공동 직무설명회 (11/16(수),14시~,온라인)
2022학년도 2학기 산학활동참여 마일리지 장학금 신청
2022학년도 2학기 SW융합전공 장학생 선발 결과 안내


In [2]:
from bs4 import BeautifulSoup
import requests

html = requests.get("https://sw7up.cbnu.ac.kr/community/notice?page=1&limit=10&sort=-createdAt").text
bsObject = BeautifulSoup(html, "html.parser")

for link in bsObject.find_all('small')[2::3]:
    print(str(link).split('>')[1].split('.<')[0])
    res['date'].append(str(link).split('>')[1].split('.<')[0])

22.11.11
22.11.11
22.11.08
22.11.07
22.10.26
22.10.24
22.08.31
22.11.11
22.11.10
22.10.27


In [3]:
%pip install selenium
%pip install webdriver_manager
%pip install firebase_admin

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get("https://sw7up.cbnu.ac.kr/community/notice?page=1&limit=10&sort=-createdAt")
time.sleep(2)

for i in range(10):
    elem = browser.find_elements(By.CLASS_NAME, 'card')
    elem[i].click()
    time.sleep(2)

    print(browser.current_url)
    res['url'].append(browser.current_url)
    browser.back()
    time.sleep(2)

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_16652\1485729119.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


https://sw7up.cbnu.ac.kr/community/notice/636df9e59c91db44543ab74f
https://sw7up.cbnu.ac.kr/community/notice/636dd0329c91db44543aa32e
https://sw7up.cbnu.ac.kr/community/notice/636a0f229c91db445439e7d9
https://sw7up.cbnu.ac.kr/community/notice/636854f6fdde24339b27af5f
https://sw7up.cbnu.ac.kr/community/notice/6358779bfdde24339b243fbc
https://sw7up.cbnu.ac.kr/community/notice/6355f03cfdde24339b23e49c
https://sw7up.cbnu.ac.kr/community/notice/630ee041f952f940ce675975
https://sw7up.cbnu.ac.kr/community/notice/636dac839c91db44543a9851
https://sw7up.cbnu.ac.kr/community/notice/636c84179c91db44543a6386
https://sw7up.cbnu.ac.kr/community/notice/6359c950fdde24339b249663


In [7]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("<your private key>")
firebase_admin.initialize_app(cred)


In [10]:
from firebase_admin import firestore

db = firestore.client()

for i in range(10):
    doc_ref = db.collection(u'sw7up').document(u'link%d' % i)
    doc_ref.set({'title': res['title'][i], 
                'date': res['date'][i],
                'url': res['url'][i]})